# Pozos en WELL package

In [1]:
# This is because I was changing mf_plots.py as I was working here
%load_ext autoreload
%autoreload 2

In [2]:
import flopy.mf6 as mf6
import pandas as pd
from pathlib import Path

In [3]:
# import mf_plots.py located in the same directory
import os
current_dir = os.getcwd()
os.environ['PYTHONPATH'] = current_dir
import mf_utils

## Localización del modelo

In [4]:
# Modflow: file and directory names
modelname = 'Moncada.nam'
model_ws = 'E:/Cursos/H2OGEO/2023_Modelacion_HGeo/Ej_3'
model_ws_path = Path(model_ws)

# Units for labels
length_unit = 'm'
time_unit = 'day'

## Read the model and get some basic characteristics

In [26]:
# gwf is instantiated and some basic features are shown
sim = mf6.MFSimulation.load(sim_name=modelname, exe_name='mf6', sim_ws=model_ws)

model_names = list(sim.model_names)
print('\nModel names in sim', model_names)

gwf = sim.get_model(model_names[0])

grid_type = gwf.modelgrid.grid_type
print('Type of grid:', grid_type)
    
disv = gwf.get_package('disv')
print('Number of layers:', disv.nlay)

tdis = sim.tdis
nper = tdis.nper.get_data()
print('Number of stress periods:', nper)   

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package disv...
    loading package ic...
    loading package npf...
    loading package gnc...
    loading package oc...
    loading package chd...
    loading package wel...
    loading package riv...
    loading package rch...
  loading solution package modflow...

Model names in sim ['modflow']
Type of grid: vertex
Number of layers: {internal}
(1)

Number of stress periods: 1


## Datos de los pozos


In [27]:
# Get the well package
well_pck = gwf.get_package('WEL')
well_df = mf_utils.wells_to_df(well_pck, False)

The number of wells in the model is: 19
Number of pumping rates (nwells*nstress_periods):  19


In [28]:
well_df.head()

,layer,cell_id,pumping_rate,stress_period,well_id
0,0,16898,-144.44,0,'productes_aditius_2'
1,0,16620,-1183.56,0,'productes_aditius_1'
2,0,9946,-155.50,0,'pozo_indianes'
3,0,8556,-65.75,0,'pozo_ceballos'
4,0,4384,-24.93,0,'pozo_cal_sant'


In [13]:
well_df.to_csv(r'E:\Cursos\H2OGEO\2023_Modelacion_HGeo\Ej_3\Pozos\Pumping_rates.csv', index=False)

## Read the file with the pumping rates

In [25]:
# Define the new pumping rates for the wells you want to change
# The keys are tuples where the first element is the well name, the second element is the stress period,
# the third element is the layer, and the fourth element is the cell identifier
# well identifier, stress period, layer, cell identifier
new_pumping_rates = {
    ("'productes_aditius_2'", 0, 0, 16898): -144.44,  # Change the pumping rate of 'well_1' in stress period 1, layer 1, cell 16899
    ("'pozo_indianes'", 0, 0, 9946): -155.5,  # Change the pumping rate of 'well_2' in stress period 2, layer 1, cell 16901
    # Add more wells here if needed
}

obj = mf_utils.wells_update_pumping_rates(well_pck, new_pumping_rates)
sim.write_simulation()

'productes_aditius_2'has been updated
'pozo_indianes'has been updated
writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package modflow...
  writing model modflow...
    writing model name file...
    writing package disv...
    writing package ic...
    writing package npf...
    writing package gnc...
    writing package oc...
    writing package chd-1...
    writing package wel-1...
    writing package riv-1...
    writing package rch-1...


In [24]:
print(obj)     

{0: rec.array([((0, 16898), -1.444400e+02, 0, "'productes_aditius_2'"),
           ((0, 16620), -1.183560e+03, 0, "'productes_aditius_1'"),
           ((0, 9946), -1.555000e+02, 0, "'pozo_indianes'"),
           ((0, 8556), -6.575000e+01, 0, "'pozo_ceballos'"),
           ((0, 4384), -2.493000e+01, 0, "'pozo_cal_sant'"),
           ((0, 14814), -5.205000e+01, 0, "'c___montiu'"),
           ((0, 19125), -1.304600e+02, 0, "'intorsa'"),
           ((0, 7030), -7.237000e+02, 0, "'gyr'"),
           ((0, 6195), -3.945000e+01, 0, "'polinter_1'"),
           ((0, 23460), -2.828050e+03, 0, "'ferrolan'"),
           ((0, 30137), -1.136990e+03, 0, "'central_besã_s_3_sgae_pozo_st__andreu'"),
           ((0, 5646), -7.562000e+01, 0, "'polinter_2'"),
           ((0, 4812), -2.169860e+03, 0, "'p_3_pla_d_en_coll'"),
           ((0, 5926), -3.288000e+01, 0, "'p_1_pla_d_en_coll'"),
           ((0, 3845), -4.419000e+01, 0, "'motor_ibã_rica'"),
           ((0, 2878), -7.237000e+02, 0, "'pozo_tuxants'"),


In [19]:
well_df.head()

,layer,cell_id,pumping_rate,stress_period,well_id
0,0,16898,-145.66,0,'productes_aditius_2'
1,0,16620,-1183.56,0,'productes_aditius_1'
2,0,9946,-155.50,0,'pozo_indianes'
3,0,8556,-65.75,0,'pozo_ceballos'
4,0,4384,-24.93,0,'pozo_cal_sant'
